In [1]:
########################################################################
# File       : CZI_ZARR.ipynb
# Version    : 0.1
# Author     : czsrh
# Date       : 12.11.2019
# Insitution : Carl Zeiss Microscopy GmbH
#
# Disclaimer: Just for testing - Use at your own risk.
# Feedback or Improvements are welcome.
########################################################################

In [1]:

# this can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# import the libraries mentioned above
from apeer_ometiff_library import io, processing, omexmlClass
import czifile as zis
import xmltodict
import os
import numpy as np
import ipywidgets as widgets
import napari
import imgfileutils as imf
import xml.etree.ElementTree as ET
import zarr

In [3]:
# define your testfiles here

#testfolder = r'C:\Users\m1srh\Documents\GitHub\ipy_notebooks\Read_OMETIFF_CZI\testdata'
testfolder = r'/datadisk1/tuxedo/Github/ipy_notebooks/Read_OMETIFF_CZI/testdata'

imgdict = {1: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_green.ome.tiff'),
           2: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_red.ome.tiff'),
           3: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small.ome.tiff'),
           4: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_green.czi'),
           5: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_red.czi'),
           6: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small.czi')
}

#filename = imgdict[6]
#filename = r'/datadisk1/tuxedo/testpictures/Testdata_Zeiss/celldivison/CellDivision_T=15_Z=20_CH=2_DCV.czi'
filename = r'L:\Data\Testdata_Zeiss\Castor\Castor_Beta\Castor_Beta2\20150610\Test4_4Ch_Hoechst_FITC_A568_MitoDR_20X0.95_1.0X_02_WellD3.czi'

In [4]:
# read metadata and array differently for OME-TIFF or CZI data
if filename.lower().endswith('.ome.tiff') or filename.lower().endswith('.ome.tif'):
    
    # Return value is an array of order (T, Z, C, X, Y)
    (array, omexml) = io.read_ometiff(filename)
    metadata, add_metadata = imf.get_metadata(filename, series=0)
    
if filename.lower().endswith('.czi'):

    # get the array and the metadata
    array, metadata, add_metadata = imf.get_array_czi(filename, return_addmd=False)

In [5]:
# show all the metadata
for key, value in metadata.items():
    # print all key-value pairs for the dictionary
    print(key, ' : ', value)

Directory  :  L:\Data\Testdata_Zeiss\Castor\Castor_Beta\Castor_Beta2\20150610
Filename  :  Test4_4Ch_Hoechst_FITC_A568_MitoDR_20X0.95_1.0X_02_WellD3.czi
Extension  :  czi
ImageType  :  czi
Name  :  None
AcqDate  :  2015-06-10T16:46:55.9482799Z
TotalSeries  :  None
SizeX  :  7164
SizeY  :  7964
SizeZ  :  1
SizeC  :  5
SizeT  :  1
Sizes BF  :  None
DimOrder BF  :  None
DimOrder BF Array  :  None
DimOrder CZI  :  {'B': 0, 'S': 1, 'T': -1, 'C': 2, 'Z': -1, 'Y': 3, 'X': 4, '0': 5}
Axes  :  BSCYX0
Shape  :  (1, 1, 5, 7964, 7164, 1)
isRGB  :  None
ObjNA  :  0.95
ObjMag  :  20.0
ObjID  :  Objective:1
ObjName  :  Plan-Apochromat 20x/0.95
ObjImmersion  :  Air
XScale  :  0.227
YScale  :  0.227
ZScale  :  0.227
XScaleUnit  :  µm
YScaleUnit  :  µm
ZScaleUnit  :  None
DetectorModel  :  Axiocam 506 #3
DetectorName  :  Axiocam506m
DetectorID  :  Detector:Axiocam 506 #3
InstrumentID  :  None
Channels  :  ['MT633', 'AF568', 'FITC', 'DAPI', 'PGC']
ImageIDs  :  []
NumPy.dtype  :  uint16
PixelType  :  Gray

In [6]:
# outout the shape of the returned numpy array

# shape of numpy array
print('Array Shape: ', array.shape)

# dimension order from metadata
print('Dimension Order (BioFormats) : ', metadata['DimOrder BF Array'])

# shape and dimension entry from CZI file as returned by czifile.py
print('CZI Array Shape : ', metadata['Shape'])
print('CZI Dimension Entry : ', metadata['Axes'])

Array Shape:  (1, 1, 5, 7964, 7164)
Dimension Order (BioFormats) :  None
CZI Array Shape :  (1, 1, 5, 7964, 7164, 1)
CZI Dimension Entry :  BSCYX0


In [8]:
z = zarr.array(array, chunks=(1, 1, 5, 7964, 7164), dtype='uint16')
zarr_info = z.info
print(type(zarr_info))

if type(z) is np.ndarray:
    print('NumPy Array')
elif type(z) is zarr.core.Array:
    print('ZARR')  

<class 'zarr.util.InfoReporter'>
ZARR


In [9]:
# display data using ipy widgets
if metadata['Extension'] == 'ome.tiff':
    ui, out = imf.create_ipyviewer_ome_tiff(array, metadata)
if metadata['Extension'] == 'czi':
    #ui, out = imf.create_ipyviewer_czi(array, metadata)
    ui, out = imf.create_ipyviewer_czi(z[:], metadata)

display(ui, out)

55 37601


Output()

In [10]:
# try to configure napari automatiaclly based on metadata
#imf.show_napari(array, metadata)
imf.show_napari(z[:], metadata)

Initializing Napari Viewer ...
Dim PosT :  -1
Dim PosZ :  -1
Dim PosC :  2
Scale Factors :  [1.0, 1.0, 1.0, 1.0, 1.0]
Shape Channel :  0 (1, 1, 7964, 7164)
Adding Channel:  MT633
Scaling Factors:  [1.0, 1.0, 1.0, 1.0, 1.0]
Scaling:  [362, 13926.0]
Shape Channel :  1 (1, 1, 7964, 7164)
Adding Channel:  AF568
Scaling Factors:  [1.0, 1.0, 1.0, 1.0, 1.0]
Scaling:  [82, 3721.0]
Shape Channel :  2 (1, 1, 7964, 7164)
Adding Channel:  FITC
Scaling Factors:  [1.0, 1.0, 1.0, 1.0, 1.0]
Scaling:  [577, 13926.0]
Shape Channel :  3 (1, 1, 7964, 7164)
Adding Channel:  DAPI
Scaling Factors:  [1.0, 1.0, 1.0, 1.0, 1.0]
Scaling:  [55, 10712.0]
Shape Channel :  4 (1, 1, 7964, 7164)
Adding Channel:  PGC
Scaling Factors:  [1.0, 1.0, 1.0, 1.0, 1.0]
Scaling:  [26413, 31961.0]


In [ ]:
# configure napari viewer manually - check array shape and dimensions order carefully 
    
# get the scalefactors
scalefactors = imf.get_scalefactor(metadata)
print(scalefactors)

array = np.squeeze(array, axis=(0, 1))

viewer = napari.Viewer()
# add every channel as a single layer
for ch in range(metadata['SizeC']):
    chname = metadata['Channels'][ch]
    viewer.add_image(array[ch, :, :, :], name=chname, scale=(1, scalefactors['zx'], 1, 1))

jupyter nbconvert CZI-ZARR.ipynb --to slides --post serve